In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F 

In [2]:
spark = (
    SparkSession.builder.appName('Aula 5 - FUNÇÕES DE TEXTO: Manipulando Strings com Funções')
    .config('spark.sql.repl.eagerEval.enabled', True)
    .getOrCreate())

In [6]:
spark

In [3]:
local_file = r'C:\Users\mluiz\Documents\05_Python\PySpark\datasets\LOGINS.parquet'

In [7]:
df = spark.read.parquet(local_file).select('email','senha','estado','cor_favorita','profissao')

In [8]:
df.show(3)

+--------------------+----------+------+------------+--------------------+
|               email|     senha|estado|cor_favorita|           profissao|
+--------------------+----------+------+------------+--------------------+
|pedro-lucas53@gma...|+7^7E%xFBc|    RR|        Roxo|    Jogador De Golfe|
|rezendeisaac@hotm...|_O_2GRnGOe|    GO|       Ciano|Atleta De Arremes...|
|felipepires@uol.c...|*Aw5EOAvy9|    MG|        Azul|      Papiloscopista|
+--------------------+----------+------+------------+--------------------+
only showing top 3 rows



In [18]:
(
    df
    # Adiciona uma coluna chamada 'usuario', que extrai o nome de usuário do e-mail dividindo-o pelo '@'.
    # Em seguida, salva esses elementos em uma lista e retorna apenas o primeiro índice dessa lista.
    # Por exemplo, se o e-mail for 'teste@teste.com', isso resultará em ['teste', 'teste.com'],
    # e o primeiro índice da lista (índice 0) será retornado, ou seja, 'teste'.
    .withColumn('usuario', F.split(df.email, '@').getItem(0))
    # Adiciona uma coluna chamada 'provedor', que extrai o provedor do e-mail dividindo-o pelo '@'.
    .withColumn('provedor', F.split(df.email, '@').getItem(1))
    # Como a coluna 'provedor' ainda não está salva em 'df', utilizamos F.col para referenciar o nome da coluna.
    # F.col é usado para acessar colunas existentes ou criadas anteriormente, por isso é preferível usá-lo.
    # O '.' (ponto) é um caractere especial, então para ser reconhecido corretamente, é necessário usar '\.'.
    # Em seguida, é criada a coluna 'nome_provedor' que pega o nome do provedor (parte antes do primeiro '.') e a adiciona ao DataFrame.
    .withColumn('nome_provedor', F.split(F.col('provedor'), '\.').getItem(0))
).show(5, False)

<>:14: SyntaxWarning: invalid escape sequence '\.'
<>:14: SyntaxWarning: invalid escape sequence '\.'
C:\Users\mluiz\AppData\Local\Temp\ipykernel_23380\1860085315.py:14: SyntaxWarning: invalid escape sequence '\.'
  .withColumn('nome_provedor', F.split(F.col('provedor'), '\.').getItem(0))


+------------------------+----------+------+------------+---------------------------+-------------+-----------+-------------+
|email                   |senha     |estado|cor_favorita|profissao                  |usuario      |provedor   |nome_provedor|
+------------------------+----------+------+------------+---------------------------+-------------+-----------+-------------+
|pedro-lucas53@gmail.com |+7^7E%xFBc|RR    |Roxo        |Jogador De Golfe           |pedro-lucas53|gmail.com  |gmail        |
|rezendeisaac@hotmail.com|_O_2GRnGOe|GO    |Ciano       |Atleta De Arremesso De Peso|rezendeisaac |hotmail.com|hotmail      |
|felipepires@uol.com.br  |*Aw5EOAvy9|MG    |Azul        |Papiloscopista             |felipepires  |uol.com.br |uol          |
|stellamoraes@bol.com.br |mw0AWYAs#s|AC    |Marrom      |Aeromoça                   |stellamoraes |bol.com.br |bol          |
|wcarvalho@ig.com.br     |pGD%!2Pq5X|AP    |Laranja     |Fonoaudiólogo              |wcarvalho    |ig.com.br  |ig     

In [26]:
(
    df
    .withColumn('concat', F.concat(df.profissao, df.cor_favorita))
    .withColumn('concat2', F.concat_ws(' ',df.profissao, df.cor_favorita))
    .withColumn('concat3', F.concat(df.estado, F.lit(' - BR')))
).show(3, False)

+------------------------+----------+------+------------+---------------------------+--------------------------------+---------------------------------+-------+
|email                   |senha     |estado|cor_favorita|profissao                  |concat                          |concat2                          |concat3|
+------------------------+----------+------+------------+---------------------------+--------------------------------+---------------------------------+-------+
|pedro-lucas53@gmail.com |+7^7E%xFBc|RR    |Roxo        |Jogador De Golfe           |Jogador De GolfeRoxo            |Jogador De Golfe Roxo            |RR - BR|
|rezendeisaac@hotmail.com|_O_2GRnGOe|GO    |Ciano       |Atleta De Arremesso De Peso|Atleta De Arremesso De PesoCiano|Atleta De Arremesso De Peso Ciano|GO - BR|
|felipepires@uol.com.br  |*Aw5EOAvy9|MG    |Azul        |Papiloscopista             |PapiloscopistaAzul              |Papiloscopista Azul              |MG - BR|
+------------------------+--------

In [29]:
(
    df
    .withColumn('lower', F.lower(df.profissao))
    .withColumn('upper', F.upper(df.cor_favorita))
    .withColumn('initcap', F.initcap(F.col('lower')))
).show(3, False)

+------------------------+----------+------+------------+---------------------------+---------------------------+-----+---------------------------+
|email                   |senha     |estado|cor_favorita|profissao                  |lower                      |upper|initcap                    |
+------------------------+----------+------+------------+---------------------------+---------------------------+-----+---------------------------+
|pedro-lucas53@gmail.com |+7^7E%xFBc|RR    |Roxo        |Jogador De Golfe           |jogador de golfe           |ROXO |Jogador De Golfe           |
|rezendeisaac@hotmail.com|_O_2GRnGOe|GO    |Ciano       |Atleta De Arremesso De Peso|atleta de arremesso de peso|CIANO|Atleta De Arremesso De Peso|
|felipepires@uol.com.br  |*Aw5EOAvy9|MG    |Azul        |Papiloscopista             |papiloscopista             |AZUL |Papiloscopista             |
+------------------------+----------+------+------------+---------------------------+---------------------------

In [34]:
(
    df
    .withColumn('substring', F.substring(df.cor_favorita, 1,3)) # Extrai os 3 primeiros caracteres da coluna 'cor_favorita'
    .withColumn('substring2', F.substring(df.cor_favorita, 2,3)) # Extrai 3 caracteres a partir do segundo caracter da coluna 'cor_favorita'
    .withColumn('substring3', F.substring(df.cor_favorita, 2,100)) # Se a coluna 'cor_favorita' tiver menos de 100 caracteres, será extraído o máximo possível a partir do segundo caracter.   
).show(5)

+--------------------+----------+------+------------+--------------------+---------+----------+----------+
|               email|     senha|estado|cor_favorita|           profissao|substring|substring2|substring3|
+--------------------+----------+------+------------+--------------------+---------+----------+----------+
|pedro-lucas53@gma...|+7^7E%xFBc|    RR|        Roxo|    Jogador De Golfe|      Rox|       oxo|       oxo|
|rezendeisaac@hotm...|_O_2GRnGOe|    GO|       Ciano|Atleta De Arremes...|      Cia|       ian|      iano|
|felipepires@uol.c...|*Aw5EOAvy9|    MG|        Azul|      Papiloscopista|      Azu|       zul|       zul|
|stellamoraes@bol....|mw0AWYAs#s|    AC|      Marrom|            Aeromoça|      Mar|       arr|     arrom|
| wcarvalho@ig.com.br|pGD%!2Pq5X|    AP|     Laranja|       Fonoaudiólogo|      Lar|       ara|    aranja|
+--------------------+----------+------+------------+--------------------+---------+----------+----------+
only showing top 5 rows



In [46]:
(
    df
    .withColumn('format_string', F.format_string('Olá %s, sua cor favorita é %s', df.email, df.cor_favorita))
    # Retorna a posição de uma substring (letra) ocupa em uma string, retorno o primeiro que encontra
    .withColumn('instr', F.instr(df.email, '@'))
    .withColumn('instr2', F.instr(df.email, 'gmail'))
    # Contabiliza quantos caracteres tem uma string
    .withColumn('length', F.length(df.profissao))
    # Repetir o valor de uma coluna por um determinado numero de vezes, no exemplo, repete o estado 3x
    .withColumn('repeat', F.repeat(df.estado, 3))
    # Aparar exesso de espaços de uma string
    .withColumn('concat', F.concat(F.lit('    '), df.estado, F.lit('    ')))
    #.withColumn('trim', F.trim())
).show(5, False)

+------------------------+----------+------+------------+---------------------------+------------------------------------------------------+-----+------+------+------+----------+
|email                   |senha     |estado|cor_favorita|profissao                  |format_string                                         |instr|instr2|length|repeat|concat    |
+------------------------+----------+------+------------+---------------------------+------------------------------------------------------+-----+------+------+------+----------+
|pedro-lucas53@gmail.com |+7^7E%xFBc|RR    |Roxo        |Jogador De Golfe           |Olá pedro-lucas53@gmail.com, sua cor favorita é Roxo  |14   |15    |16    |RRRRRR|    RR    |
|rezendeisaac@hotmail.com|_O_2GRnGOe|GO    |Ciano       |Atleta De Arremesso De Peso|Olá rezendeisaac@hotmail.com, sua cor favorita é Ciano|13   |0     |27    |GOGOGO|    GO    |
|felipepires@uol.com.br  |*Aw5EOAvy9|MG    |Azul        |Papiloscopista             |Olá felipepires@uol.

In [47]:
(
    df
    # Aparar exesso de espaços de uma string
    .withColumn('concat', F.concat(F.lit('    '), df.estado, F.lit('    ')))
    .withColumn('trim', F.trim(F.col('concat')))
).show(5, False)

+------------------------+----------+------+------------+---------------------------+----------+----+
|email                   |senha     |estado|cor_favorita|profissao                  |concat    |trim|
+------------------------+----------+------+------------+---------------------------+----------+----+
|pedro-lucas53@gmail.com |+7^7E%xFBc|RR    |Roxo        |Jogador De Golfe           |    RR    |RR  |
|rezendeisaac@hotmail.com|_O_2GRnGOe|GO    |Ciano       |Atleta De Arremesso De Peso|    GO    |GO  |
|felipepires@uol.com.br  |*Aw5EOAvy9|MG    |Azul        |Papiloscopista             |    MG    |MG  |
|stellamoraes@bol.com.br |mw0AWYAs#s|AC    |Marrom      |Aeromoça                   |    AC    |AC  |
|wcarvalho@ig.com.br     |pGD%!2Pq5X|AP    |Laranja     |Fonoaudiólogo              |    AP    |AP  |
+------------------------+----------+------+------------+---------------------------+----------+----+
only showing top 5 rows



In [52]:
(
    df
    # Preenche a coluna 'cor_favorita' com pontos (.) à esquerda até atingir um total de 10 caracteres
    .withColumn('lpad', F.lpad(df.cor_favorita, 10, '.'))
    # Preenche a coluna 'cor_favorita' com pontos (.) à direita até atingir um total de 10 caracteres
    .withColumn('rpad', F.rpad(df.cor_favorita, 10, '.'))
).show(5, False)

+------------------------+----------+------+------------+---------------------------+----------+----------+
|email                   |senha     |estado|cor_favorita|profissao                  |lpad      |rpad      |
+------------------------+----------+------+------------+---------------------------+----------+----------+
|pedro-lucas53@gmail.com |+7^7E%xFBc|RR    |Roxo        |Jogador De Golfe           |......Roxo|Roxo......|
|rezendeisaac@hotmail.com|_O_2GRnGOe|GO    |Ciano       |Atleta De Arremesso De Peso|.....Ciano|Ciano.....|
|felipepires@uol.com.br  |*Aw5EOAvy9|MG    |Azul        |Papiloscopista             |......Azul|Azul......|
|stellamoraes@bol.com.br |mw0AWYAs#s|AC    |Marrom      |Aeromoça                   |....Marrom|Marrom....|
|wcarvalho@ig.com.br     |pGD%!2Pq5X|AP    |Laranja     |Fonoaudiólogo              |...Laranja|Laranja...|
+------------------------+----------+------+------------+---------------------------+----------+----------+
only showing top 5 rows

